## Asteroid conjunction finder

Find close approaches between asteroid positions (obtained dynamically from MPC ephemeris service) and NGC DSO catalog (cached locally from Vizier table)

In [2]:
# Parameters, edit to taste

# Date & time for ephemeris lookup

OBS_TIME = '2025-02-15 20:00:00'

# coords of home location (used to calculate altitude of conjunction)

OBS_LAT = 52.9
OBS_LONG = 0.5

# minimum altitude for conjunctions (anything below is filtered out for visibility)
MIN_VISIBLE_ALT = 10 # degrees

# mimimum angular size of DSO to be visible in image
DSO_MIN_SIZE = 2  # 5% of Seestar S50 FoV width

# maximum magnitude for DSO to be visible 
DSO_MAX_MAG = 15  # approx limiting magnitude for Seestar S50

# Max angular distance between asteroid and DSO
MAX_SEPARATION_ARCMIN = 40 # approx frame width in Seestar S50

# arcsec per pixel for scope (used to convert proper motion to px per hr
ARCSEC_PER_PIXEL = 2.49 # seestar S50

# number of asteroids to check (from list ordered by absolute magnitude)
CHECK_ASTEROIDS = 100

In [ ]:
import os

if os.getenv("COLAB_RELEASE_TAG"):
  from google.colab import drive
  drive.mount('/content/drive')
  AST_PATH = '/content/drive/My Drive/Astronomy/asteroids with abs mag below 10.csv'
  NGC_PATH = '/content/drive/My Drive/Astronomy/ngc_catalog.ecsv'
  !pip install astroquery
else:
  AST_PATH = './asteroids with abs mag below 10.csv'
  NGC_PATH = './ngc_catalog.ecsv'

In [ ]:
# load list of asteroids to search from file (ordered by absolute magnitude)

import csv

with open(AST_PATH) as csvfile:
    csv_reader = csv.reader(csvfile)
    next(csv_reader) # skip header row
    asteroid_names = [row[3] for _, row in zip(range(CHECK_ASTEROIDS), csv_reader)]

In [ ]:
# Fetch ngc catalog from CDS vizier server
# This code only needs to be run once to cache the table locally

# from astroquery.vizier import Vizier

# vizier = Vizier()
# vizier.ROW_LIMIT = -1

# ngc_tables = vizier.get_catalogs('VII/118/ngc2000')

# # save ngc table locally for future reference

# ngc_tables[0].write('./ngc_catalog.ecsv')


In [ ]:
# create SkyCoord catalog of ngc DSOs for searching

from astropy.io import ascii

# load locally cached ngc table 
ngc_table = ascii.read(NGC_PATH)

# filter table to size and magnitude limits

filtered_ngc_table = ngc_table[ngc_table['size'] > DSO_MIN_SIZE]
filtered_ngc_table = filtered_ngc_table[filtered_ngc_table['mag'] < DSO_MAX_MAG]

In [4]:
from astropy.coordinates import SkyCoord
import astropy.units as u

ngc_catalog = SkyCoord(ra = filtered_ngc_table['RAB2000'], dec = filtered_ngc_table['DEB2000'],
                       unit = (u.hourangle, u.deg))



In [5]:
# Query MPC Horizons service for asteroid ephemeris data

from astroquery.mpc import MPC
from astropy.table import vstack

ephemerides = [MPC.get_ephemeris(a, start = OBS_TIME, number = 1) for a in asteroid_names]
ephemerides = vstack(ephemerides)

# create SkyCoord catalog for search
ephemerides_cat = SkyCoord(ra = ephemerides['RA'], dec = ephemerides['Dec'], unit = u.deg)

In [ ]:
# Find sky conjunctions of asteroid positions and NGC catalog

from astropy.coordinates import AltAz, EarthLocation

import warnings
warnings.filterwarnings('ignore') # block formatting warning message

idx_ngc, idx_asteroid, sep2d, _ = ephemerides_cat.search_around_sky(ngc_catalog, seplimit = MAX_SEPARATION_ARCMIN * u.arcmin)

home = EarthLocation(lat = OBS_LAT * u.deg, lon = OBS_LONG * u.deg)
home_altaz = AltAz(obstime = OBS_TIME, location = home)

print('Predicted conjunctions for ', OBS_TIME, '\n')
for ingc, ia, sep in zip(idx_ngc, idx_asteroid, sep2d):
    ngc = filtered_ngc_table[ingc]
    ngc_loc = ngc_catalog[ingc]
    ephem = ephemerides[ia]
    ephem_coords = ephemerides_cat[ia]
    ngc_mag = filtered_ngc_table[ingc]['mag']
    ast_mag = ephem['V']
    dist = ephem['Delta']
    pm = ephem['Proper motion'] # arcsec per hour
    pm_px = pm/ARCSEC_PER_PIXEL  #px per hour
    altaz_ast =  ephem_coords.transform_to(home_altaz).alt.degree
    altaz_ngc = ngc_loc.transform_to(home_altaz).alt.degree
    skip_count = 0
    name = ngc['Name']
    if name[0] != 'I':
        name = 'NGC ' + name
    if altaz_ast < MIN_VISIBLE_ALT or altaz_ngc < MIN_VISIBLE_ALT:
        skip_count += 1
        continue
    print(f"{name}, {ngc['Type']}, (alt={altaz_ngc:.1f}, mag={ngc_mag:.1f}) <> {asteroid_names[ia]} (alt={altaz_ast:.1f}, mag={ast_mag:.1f}), sep={sep:.2f}, pm={pm:.1f}\"({pm_px:.1f}px)/hr, dist={dist})")
print('\nThere were', skip_count, 'conjunctions below the visibility threshold')
    

Predicted conjunctions for  2025-02-05 20:00:00 

NGC 660, Gx, (alt=34.1, mag=10.8) <> Hygiea (alt=34.2, mag=12.0), sep=0.37 deg, pm=32.4"(13.0px)/hr, dist=3.659)
NGC 2355, OC, (alt=43.0, mag=10.0) <> Eurynome (alt=43.5, mag=10.9), sep=0.60 deg, pm=23.3"(9.4px)/hr, dist=1.316)

There were 1 conjunctions below the visibility threshold


: 